In [24]:
import requests
import pandas as pd
import configparser  
config = configparser.ConfigParser() 
config.read("C:/Users/tomas/Dropbox/Prop-Trading Business/oanda.cfg") 

api_key         =  config["oanda"]['api_key']
oanda_api_url   =  config["oanda"]['oanda_api_url']
account_id      =  config["oanda"]['account_id']
headers         = {'Authorization': f'Bearer {api_key}'}
my_url          = "https://api-fxpractice.oanda.com"
session         = requests.Session()

In [41]:
# My functions for api requests
def GetAccount(headers,account_id):
    """
    Function type: Api Data Request
    Get the full details for a single Account that client has access to.
    Full pending Order, open Trade and open Position representations are provided
    """
    try:
        response = session.get(f"{my_url}/v3/accounts/"+account_id,headers=headers)
        data = response.json()
        print(response.status_code)
    except:
        print(response.status_code)
    return data

def GetAccountSummary(headers,account_id):
    """
    Function type: Api Data Request
    Get a summary for single Account that client has access to.
    """
    try:
        response = session.get(f"{my_url}/v3/accounts/"+account_id+"/summary",headers=headers)
        data = response.json()
        print(response.status_code)
    except:
        print(response.status_code)
    return data

def GetAllInstruments(headers,account_id):
    """
    Function type: Api Data Request
    Get list of tradable instruments for the given Account.
    """
    try:
        response = session.get(f"{my_url}/v3/accounts/"+account_id+"/instruments",headers=headers)
        data = response.json()
        print(response.status_code)
    except:
        print(response.status_code)
    return data

def GetInstrument(headers,account_id,symbol):
    """
    Function type: Api Data Request
    Get list of tradable instruments for the given Account.
    """
    try:
        response = session.get(f"{my_url}/v3/accounts/"+account_id+"/instruments",headers=headers,params={"instruments":symbol})
        data = response.json()
        print(response.status_code)
    except:
        print(response.status_code)
    return data

def GetCandleData(headers,symbol,granularity,price,count):
    """
    Function type: Api Data Request
    Fetch candlestick data for an instrument
    """
    try:
        response = session.get(f"{my_url}/v3/instruments/"+symbol+"/candles",headers=headers, params={"granularity":granularity,"price":price,"count":count })
        data = response.json()
        print(response.status_code)
    except:
        print(response.status_code)
    return data

def GetCandlesDataFrame(json_response):
    prices = ["mid", "bid", "ask"]
    ohlc = ["o","h","l","c"]
    our_data = []
    for candle in json_response["candles"]:
        if candle["complete"] == False:
            continue
        new_dict = {}
        new_dict["time"] = candle["time"]
        new_dict["volume"] = candle["volume"]
        for price in prices:
            for oh in ohlc:
                new_dict[f"{price}_{oh}"] = candle[price][oh]
        our_data.append(new_dict)
    candles_df = pd.DataFrame(our_data)
    return candles_df

def SaveFile(candles_df, symbol, granularity):
    """
    Function type: Data Manipulation
    This funcion save historical data to file for detailed analysis
    """
    candles_df.to_csv(f"historical_data/{symbol}_{granularity}.csv")

def CreateData(headers ,symbol , granularity, price, count):
    json_data = GetCandleData(headers, symbol ,granularity ,price, count)

    df = GetCandlesDataFrame(json_data)
    print(f"{symbol} downloaded {df.shape[0]} candles from {df.time.min()} to {df.time.max()}")
    SaveFile(df, symbol, granularity)
    print(f"Data for {symbol} was saved do file")

In [42]:
CreateData(headers ,"EUR_USD" , granularity="H1",price="MBA",count=1000)

200
EUR_USD downloaded 999 candles from 2021-09-06T07:00:00.000000000Z to 2021-11-02T21:00:00.000000000Z
Data for EUR_USD was saved do file


In [27]:
d = GetInstrument(headers,account_id,"EUR_USD")
p = GetInstrument(headers,account_id,"EUR_USD")["instruments"]
print(p[0]["name"])
d.keys()

200
200
EUR_USD


dict_keys(['instruments', 'lastTransactionID'])

In [28]:
#GetAccount(ao.headers,ao.account_id)
#GetCandleData(ao.headers,"EUR_USD",granularity="H1",price="MBA",count=10)

In [29]:
instruments = GetAllInstruments(headers,account_id)
my_list = instruments["instruments"]
my_list[0].keys()

200


dict_keys(['name', 'type', 'displayName', 'pipLocation', 'displayPrecision', 'tradeUnitsPrecision', 'minimumTradeSize', 'maximumTrailingStopDistance', 'minimumTrailingStopDistance', 'maximumPositionSize', 'maximumOrderUnits', 'marginRate', 'guaranteedStopLossOrderMode', 'tags', 'financing'])

In [30]:
# get data from instruments dictionary an save in new one in my list
my_list = instruments["instruments"]

my_instruments = list()
for i in my_list:
    new = dict(
        name = i["name"],
        type = i["type"],
        displayName = i["displayName"],
        marginRate = i["marginRate"],
        displayPrecision = i["displayPrecision"],
        minimumTradeSize = i["minimumTradeSize"]
    )
    my_instruments.append(new)

In [31]:
# Save data to dataframe for later use
my_instruments = pd.DataFrame(my_instruments)
#my_instruments.to_csv("Oanda_instrument_list.csv")
my_instruments

,name,type,displayName,marginRate,displayPrecision,minimumTradeSize
0,EUR_HUF,CURRENCY,EUR/HUF,0.05,3,1
1,FR40_EUR,CFD,France 40,0.05,1,0.1
2,EUR_DKK,CURRENCY,EUR/DKK,0.1,5,1
3,XAU_CHF,METAL,Gold/CHF,0.05,3,1
4,XPD_USD,CFD,Palladium,0.1,3,1
...,...,...,...,...,...,...
118,USD_PLN,CURRENCY,USD/PLN,0.05,5,1
119,AU200_AUD,CFD,Australia 200,0.05,1,0.1
120,CAD_HKD,CURRENCY,CAD/HKD,0.1,5,1
121,US30_USD,CFD,US Wall St 30,0.05,1,0.1


In [32]:
# get candlestick data
candle_data = GetCandleData(headers,"EUR_USD",granularity="H1",price="MBA",count=10)
candle_data

200


{'instrument': 'EUR_USD',
 'granularity': 'H1',
 'candles': [{'complete': True,
   'volume': 5416,
   'time': '2021-11-02T13:00:00.000000000Z',
   'bid': {'o': '1.16064', 'h': '1.16070', 'l': '1.15882', 'c': '1.15906'},
   'mid': {'o': '1.16070', 'h': '1.16077', 'l': '1.15890', 'c': '1.15913'},
   'ask': {'o': '1.16077', 'h': '1.16084', 'l': '1.15897', 'c': '1.15920'}},
  {'complete': True,
   'volume': 6480,
   'time': '2021-11-02T14:00:00.000000000Z',
   'bid': {'o': '1.15905', 'h': '1.15980', 'l': '1.15816', 'c': '1.15907'},
   'mid': {'o': '1.15912', 'h': '1.15986', 'l': '1.15822', 'c': '1.15914'},
   'ask': {'o': '1.15918', 'h': '1.15993', 'l': '1.15828', 'c': '1.15921'}},
  {'complete': True,
   'volume': 5735,
   'time': '2021-11-02T15:00:00.000000000Z',
   'bid': {'o': '1.15910', 'h': '1.15989', 'l': '1.15771', 'c': '1.15799'},
   'mid': {'o': '1.15916', 'h': '1.15995', 'l': '1.15780', 'c': '1.15806'},
   'ask': {'o': '1.15922', 'h': '1.16001', 'l': '1.15788', 'c': '1.15812'}},

In [33]:
# organize price data to data frame for analysis






In [34]:
instr_df = pd.read_csv("Oanda_instrument_list.csv")
instr_df.name.unique()

array(['EUR_HUF', 'FR40_EUR', 'EUR_DKK', 'XAU_CHF', 'XPD_USD', 'BCO_USD',
       'IN50_USD', 'NATGAS_USD', 'USD_MXN', 'GBP_USD', 'CAD_CHF',
       'EUR_GBP', 'XAG_SGD', 'JP225_USD', 'XAG_CAD', 'CN50_USD',
       'EU50_EUR', 'GBP_CHF', 'USD_THB', 'USD_ZAR', 'XAU_USD', 'XPT_USD',
       'XAU_EUR', 'EUR_NOK', 'DE30_EUR', 'GBP_PLN', 'AUD_NZD', 'XAU_HKD',
       'EUR_HKD', 'CHF_JPY', 'USD_SEK', 'GBP_SGD', 'GBP_HKD', 'EUR_NZD',
       'XAG_AUD', 'WTICO_USD', 'XAG_NZD', 'SGD_CHF', 'AUD_SGD', 'EUR_JPY',
       'TWIX_USD', 'USD_CHF', 'USD_TRY', 'GBP_JPY', 'EUR_CZK', 'BTC_USD',
       'EUR_TRY', 'USD_JPY', 'GBP_ZAR', 'SGD_JPY', 'XAU_SGD', 'USD_CZK',
       'XAU_JPY', 'XAG_JPY', 'ZAR_JPY', 'TRY_JPY', 'USD_DKK', 'EUR_PLN',
       'SUGAR_USD', 'AUD_CAD', 'USD_HKD', 'CAD_SGD', 'US2000_USD',
       'DE10YB_EUR', 'USB02Y_USD', 'USB05Y_USD', 'HK33_HKD', 'USB10Y_USD',
       'GBP_NZD', 'USD_CNH', 'XAU_NZD', 'NZD_USD', 'XAG_GBP', 'XAG_EUR',
       'CORN_USD', 'XAG_USD', 'UK100_GBP', 'NZD_HKD', 'USD_NOK',

In [35]:
# Create list of tradables to download datasets 
currency = ["EUR","USD","GBP","JPY","CHF","NZD","CAD"]

for p1 in currency:
    for p2 in currency:
        pair = f"{p1}_{p2}"
        if pair in instr_df.name.unique():
            print(pair)


EUR_USD
EUR_GBP
EUR_JPY
EUR_CHF
EUR_NZD
EUR_CAD
USD_JPY
USD_CHF
USD_CAD
GBP_USD
GBP_JPY
GBP_CHF
GBP_NZD
GBP_CAD
CHF_JPY
NZD_USD
NZD_JPY
NZD_CHF
NZD_CAD
CAD_JPY
CAD_CHF
